<a href="https://colab.research.google.com/github/evsemenova/Python-for-Analysts/blob/HW_3/%D0%A3%D1%80%D0%BE%D0%BA_3_%D0%A0%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%D1%81_%D0%B1%D0%B8%D0%B1%D0%BB%D0%B8%D0%BE%D1%82%D0%B5%D0%BA%D0%BE%D0%B9_Pandas%2C_%D1%87%D0%B0%D1%81%D1%82%D1%8C_2_%D0%9C%D0%B0%D0%BD%D0%B8%D0%BF%D1%83%D0%BB%D1%8F%D1%86%D0%B8%D0%B8_%D1%81_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%BC%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Использовать датасет с текущего урока и построить сегментированную воронку конверсии пользователей интернет-магазина по полу.**

In [ ]:
# Импортируем нужные бибилиотеки
import pandas as pd

In [ ]:
# Загружаем в Python с помощью Pandas все датасеты
user_table = pd.read_csv('user_table.csv')
home_page = pd.read_csv('home_page_table.csv')
payment_confirmation = pd.read_csv('payment_confirmation_table.csv')
payment_page = pd.read_csv('payment_page_table.csv')
search_page = pd.read_csv('search_page_table.csv')

In [ ]:
# Просматриваем первые 5 строк датасетов
user_table.head()

,user_id,date,device,sex
0,450007,2015-02-28,Desktop,Female
1,756838,2015-01-13,Desktop,Male
2,568983,2015-04-09,Desktop,Male
3,190794,2015-02-18,Desktop,Female
4,537909,2015-01-15,Desktop,Male


In [ ]:
home_page.head()

,user_id,page
0,313593,home_page
1,468315,home_page
2,264005,home_page
3,290784,home_page
4,639104,home_page


In [ ]:
payment_confirmation.head()

,user_id,page
0,123100,payment_confirmation_page
1,704999,payment_confirmation_page
2,407188,payment_confirmation_page
3,538348,payment_confirmation_page
4,841681,payment_confirmation_page


In [ ]:
payment_page.head()

,user_id,page
0,253019,payment_page
1,310478,payment_page
2,304081,payment_page
3,901286,payment_page
4,195052,payment_page


In [ ]:
search_page.head()

,user_id,page
0,15866,search_page
1,347058,search_page
2,577020,search_page
3,780347,search_page
4,383739,search_page


In [ ]:
# Просматриваем кол-во строк и столбцов каждого датасета
# Как в одном запросе добавить просмотр всех датасетов? 
# Запрос user_table.shape, home_page.shape, payment_confirmation.shape, payment_page.shape, search_page.shape 
# Выводит не читабельный вид

user_table.shape, home_page.shape

((90400, 4), (90400, 2))

In [ ]:
payment_confirmation.shape, payment_page.shape

((452, 2), (6030, 2))

In [ ]:
search_page.shape

(45200, 2)

In [ ]:
# Меняем название одинаковой колонки в каждом датасете
home_page = home_page.rename(columns = {'page':'home_page'})
search_page = search_page.rename(columns = {'page':'search_page'})
payment_page = payment_page.rename(columns = {'page':'payment_page'})
payment_confirmation = payment_confirmation.rename(columns = {'page':'payment_confirmation'})

In [ ]:
# Склеиваем последовательно все датасеты с помощью метода merge по идентификатору user_id
df = user_table.merge(home_page, on = 'user_id', how = 'left').merge(search_page, on = 'user_id', how = 'left') \
.merge(payment_page, on = 'user_id', how = 'left').merge(payment_confirmation, on = 'user_id', how = 'left')
df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,home_page,NaN,NaN,NaN
1,756838,2015-01-13,Desktop,Male,home_page,NaN,NaN,NaN
2,568983,2015-04-09,Desktop,Male,home_page,search_page,NaN,NaN
3,190794,2015-02-18,Desktop,Female,home_page,search_page,NaN,NaN
4,537909,2015-01-15,Desktop,Male,home_page,NaN,NaN,NaN


In [ ]:
df.shape

(90400, 8)

In [ ]:
# Для удоства последующих расчетов, перекодируем названия событий в 0/1 с помощью лямбда-функции
df['home_page'] = df['home_page'].apply(lambda x: 1 if x == 'home_page' else 0)
df['search_page'] = df['search_page'].apply(lambda x: 1 if x == 'search_page' else 0)
df['payment_page'] = df['payment_page'].apply(lambda x: 1 if x == 'payment_page' else 0)
df['payment_confirmation'] = df['payment_confirmation'].apply(lambda x: 1 if x == 'payment_confirmation_page' else 0)

df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,1,0,0,0
1,756838,2015-01-13,Desktop,Male,1,0,0,0
2,568983,2015-04-09,Desktop,Male,1,1,0,0
3,190794,2015-02-18,Desktop,Female,1,1,0,0
4,537909,2015-01-15,Desktop,Male,1,0,0,0


In [ ]:
# Агрегируем данные: подсчитываем количество пользователей по полу
df['sex'].value_counts()

Male      45325
Female    45075
Name: sex, dtype: int64

In [ ]:
# Подготовим группировку данных по совершению событий на сайте в разрезе пола, используя метод groupby

dfg = df.groupby('sex')[['home_page', 'search_page', 'payment_page', 'payment_confirmation']].sum().unstack('sex').unstack('sex').reset_index()

# Уберем ненужный заголовок верхнего уровня
dfg.columns.name = None

# Переименуем стандартные после процедуры заголовки
dfg.columns.name = None
dfg = dfg.rename(columns = {'index':'Action'})


# Посмотрим, что получилось
dfg.head()

,Action,Female,Male
0,home_page,45075,45325
1,search_page,22676,22524
2,payment_page,3100,2930
3,payment_confirmation,241,211


In [ ]:
# Строим воронку конверсий на сайте с сегментацией по полу пользователей
# Импортируем графическую библиотеку Plotly (https://plotly.com/python/funnel-charts/)
from plotly import graph_objects as go

# Настраиваем скрипт библиотеки Plotly нашими параметрами
fig = go.Figure()

# Часть графика для Female
fig.add_trace(go.Funnel(
    y = dfg['Action'],
    x = dfg['Female'],
    name = 'Female',
    constraintext='outside',
    textposition = 'inside',
    textinfo = 'value+percent previous'
    ))

# Часть графика для Male	
fig.add_trace(go.Funnel(
    y = dfg['Action'],
    x = dfg['Male'],
    name = 'Male',
    textposition = 'auto',
    textinfo = 'value+percent previous'
   ))

fig.update_layout(
    title = 'Воронка конверсии пользователей интернет-магазина по полу.',
    title_x=0.5,
    autosize=False,
    width=1100,
    height=500)

fig.show()

**2. Создать 2 новые фичи на основе колонки “date”: месяц и день недели (пример https://stackoverflow.com/a/62624729, https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.weekday.html ).**

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.month
df.head(10)

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation,month,day_of_week
0,450007,2015-02-28,Desktop,Female,1,0,0,0,2,Saturday
1,756838,2015-01-13,Desktop,Male,1,0,0,0,1,Tuesday
2,568983,2015-04-09,Desktop,Male,1,1,0,0,4,Thursday
3,190794,2015-02-18,Desktop,Female,1,1,0,0,2,Wednesday
4,537909,2015-01-15,Desktop,Male,1,0,0,0,1,Thursday
5,993454,2015-03-03,Desktop,Male,1,0,0,0,3,Tuesday
6,377150,2015-04-15,Desktop,Female,1,1,0,0,4,Wednesday
7,137258,2015-01-27,Mobile,Male,1,0,0,0,1,Tuesday
8,608249,2015-04-12,Desktop,Female,1,0,0,0,4,Sunday
9,615615,2015-02-24,Mobile,Female,1,1,0,0,2,Tuesday


In [ ]:
df['day_of_week'] = df['date'].dt.dayofweek 
df['day_of_week'] = df['day_of_week'].apply(lambda x: 
                                            'Monday' if x == 0
                                            else 
                                            'Tuesday' if x == 1
                                            else
                                            'Wednesday' if x == 2
                                            else
                                            'Thursday' if x == 3
                                            else
                                            'Friday' if x == 4
                                            else
                                            'Saturday' if x == 5
                                            else
                                            'Sunday' if x == 6
                                            else 'no data'
                                            )
df.head(15)

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation,month,day_of_week
0,450007,2015-02-28,Desktop,Female,1,0,0,0,2,Saturday
1,756838,2015-01-13,Desktop,Male,1,0,0,0,1,Tuesday
2,568983,2015-04-09,Desktop,Male,1,1,0,0,4,Thursday
3,190794,2015-02-18,Desktop,Female,1,1,0,0,2,Wednesday
4,537909,2015-01-15,Desktop,Male,1,0,0,0,1,Thursday
5,993454,2015-03-03,Desktop,Male,1,0,0,0,3,Tuesday
6,377150,2015-04-15,Desktop,Female,1,1,0,0,4,Wednesday
7,137258,2015-01-27,Mobile,Male,1,0,0,0,1,Tuesday
8,608249,2015-04-12,Desktop,Female,1,0,0,0,4,Sunday
9,615615,2015-02-24,Mobile,Female,1,1,0,0,2,Tuesday


**3. Определить самый топовый по продажам месяц и день недели с помощью базовых методов агрегации (sum или count).**

In [ ]:
# Определим сколько уникальных месяцев
df['month'].nunique()

4

In [ ]:
# Посчитаем кол-во платежей по месяцам
df['month'].loc[df['payment_confirmation']==1].value_counts()

1    189
2    173
4     46
3     44
Name: month, dtype: int64

In [ ]:
# Посчитаем кол-во платежей по дням недели
df['day_of_week'].loc[df['payment_confirmation']==1].value_counts()

Monday       77
Saturday     70
Tuesday      69
Thursday     66
Sunday       62
Wednesday    54
Friday       54
Name: day_of_week, dtype: int64

**4. Выяснить, в какой день недели лучше покупают женщины, а в какой мужчины с помощью одного из методов построения сводных таблиц (groupby, pivot_table или crosstab).**

In [ ]:
df.groupby(['sex', 'day_of_week'])['payment_confirmation'].count().unstack('day_of_week')

day_of_week,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
sex,,,,,,,
Female,6328,6400,6344,6506,6728,6410,6359
Male,6422,6555,6457,6416,6716,6287,6472
